In [1]:
!pip install onnxruntime-gpu

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 115.3 MB 81.9 MB/s eta 0:00:01    |███████▏                        | 25.8 MB 40.2 MB/s eta 0:00:03
     |████████████████████████████████| 15.7 MB 69.3 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 46.2 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 38.7 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 61.3 MB/s eta 0:00:01
     |████████████████████████████████| 532 kB 69.1 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Uninstalling numpy-1.21.4:
      Successfully uninstalled numpy-1.21.4


In [ ]:
import onnxruntime as ort
import torch
import mmcv
from mmcv.parallel import collate, scatter
from mmcls.datasets.pipelines import Compose
import numpy as np

session = ort.InferenceSession('resnet18.onnx',
                               providers=['CUDAExecutionProvider'])

cfg = mmcv.Config.fromfile('resnet18_8xb32_in1k.py')
test_pipeline = Compose(cfg.data.test.pipeline)
input_data = dict(img_info=dict(filename='daisy.webp'), img_prefix=None)
input_data = test_pipeline(input_data)
input_data = collate([input_data], samples_per_gpu=1)
input_data = scatter(input_data, [torch.device('cuda:0')])[0]
input_data = input_data['img']

io_binding = session.io_binding()
io_binding.bind_input(name='input',
                      device_type='cuda',
                      device_id=0,
                      element_type=np.float32,
                      shape=input_data.shape,
                      buffer_ptr=input_data.data_ptr())
io_binding.bind_output('output')

session.run_with_iobinding(io_binding)
scores = io_binding.copy_outputs_to_cpu()[0]

pred_score = np.max(scores, axis=1)[0]
pred_label = np.argmax(scores, axis=1)[0]
print(pred_score, pred_label)

/environment/miniconda3/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
